# Customized Matplotlib Color Maps

In this section are some functions that create matplotlib color maps based on chosed theme colors.

In [25]:
# First, load the codes that convert between RGB and HSL codes
!curl -O https://raw.githubusercontent.com/Leejere/python-visualization-preset/main/convert_colors.ipynb

%run convert_colors.ipynb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  9533  100  9533    0     0  83180      0 --:--:-- --:--:-- --:--:-- 83622


First, let's create a monochromatic sequential color map based off of a theme color and transitioning towards white, black, or transparent.

In [ ]:
# Let's say we use this salmon color (#EF8872)

# Inputs:
# theme_color: string, hex: the main color that this cmap is based on
# end_saturation (optional): "same", "down", "gray", or integer (pct)
# end_lightness: integer (pct); black if 0, white if 100; or "same"
# end_alpha: integer(transparency pct)
def create_mono_cmap(theme_color, end_saturation, end_lightness, end_alpha):
    
    # First, convert this color into HSL
    this_h, this_s, this_l = [i for i in hex_to_hsl(theme_color)]
    this_s /= 100
    this_l /= 100
    
    # Get end HSL
    # Keep hue same
    # Saturation
    if end_saturation == 'same':
        end_s = this_s
    elif end_saturation == 'down':
        end_s = this_s / 2
    elif end_saturation == 'gray':
        end_s = 0
    elif end_saturation <= 100 & end_saturation >= 0:
        end_s = end_saturation / 100
    else:
        print('end_saturation input error')
    
    # Lightness
    if end_lightness == 'same':
        end_l = this_l
    elif end_lightness >= 0 & end_lightness <= 100:
        end_l = end_lightness / 100
    else:
        print('end_lightness input error')
    
    # Alpha
    if end_alpha >= 0 & end_alpha <= 100:
        end_a = end_alpha / 100
    else:
        print('end_alpha input error')
    
    # Now we'll convert the hsl's into rgb's
    
    
    # Create color map
    
    vals = np.ones((256, 4)) # Create a two-dimensional np.array
    vals[:, 0] = np.linspace(this_h / 256, 1, 256) # Hue
    vals[:, 1] = np.linspace(39/256, 1, N)
    vals[:, 2] = np.linspace(41/256, 1, N)
    vals[:, 2] = np.linspace(41/256, 1, N)
    newcmp = ListedColormap(vals)

In [28]:
hsl_to_rgb(34, 12, 12)

NameError: name 'hsl_to_rgb' is not defined

In [16]:
this_hue, this_saturation, this_lightness = [i for i in hex_to_hsl('EF8872')]

In [18]:
this_lightness

49